The MNIST handwritten digit classification problem is a standard dataset used in Computer Vision and Deep Learing Models.
Here we will harness the power of Neural Networks to build a deep learning model and train it for recognising __Handwritten Digits__. <br>
__NOTE__:The recognised digit is shown as a plot(digits vs probability).


In [7]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch import optim

#Defining a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
            transforms.Normalize((0.5,),(0.5,)),                    
            ])
#Downloading and training the dataset
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train= True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle = True)

In [8]:
#Building a feed-forward model

model = nn.Sequential(nn.Linear(784,128),
                     nn.ReLU(),
                     nn.Linear(128,64),
                     nn.ReLU(),
                     nn.Linear(64,10),
                     nn.LogSoftmax(dim = 1))
#Defining Loss
criterion = nn.NLLLoss()

#Getting the data
images, lables = next(iter(trainloader))

#Flatening the images
images = images.view(images.shape[0], -1)

#forward pass and get our probabilities(because of Softmax use in model)
logpbs = model(images)

#Calculating loss
loss = criterion(logpbs, lables)
print('Before Backward:\n', model[0].weight.grad)
loss.backward()
print('After Backward:\n', model[0].weight.grad)

Before Backward:
 None
After Backward:
 tensor([[ 2.4983e-03,  2.4983e-03,  2.4983e-03,  ...,  2.4983e-03,
          2.4983e-03,  2.4983e-03],
        [-1.7430e-04, -1.7430e-04, -1.7430e-04,  ..., -1.7430e-04,
         -1.7430e-04, -1.7430e-04],
        [-4.1464e-03, -4.1464e-03, -4.1464e-03,  ..., -4.1464e-03,
         -4.1464e-03, -4.1464e-03],
        ...,
        [-3.7097e-05, -3.7097e-05, -3.7097e-05,  ..., -3.7097e-05,
         -3.7097e-05, -3.7097e-05],
        [-5.6831e-03, -5.6831e-03, -5.6831e-03,  ..., -5.6831e-03,
         -5.6831e-03, -5.6831e-03],
        [ 7.6225e-04,  7.6225e-04,  7.6225e-04,  ...,  7.6225e-04,
          7.6225e-04,  7.6225e-04]])


In [14]:
print('Initial weights - ', model[0].weight)

optimizer = optim.SGD(model.parameters(), lr = 0.01)


images, labels = next(iter(trainloader))

images = images.view(images.shape[0], -1)

optimizer.zero_grad()

logpbs = model(images)
loss = criterion(logpbs, labels)
loss.backward()
print('Final weights - ', model[0].weight.grad)

optimizer.step()


Initial weights -  Parameter containing:
tensor([[-0.0355, -0.0229, -0.0150,  ...,  0.0268, -0.0013,  0.0128],
        [ 0.0008, -0.0145, -0.0158,  ..., -0.0068, -0.0162, -0.0176],
        [-0.0126,  0.0258,  0.0253,  ..., -0.0285, -0.0033, -0.0321],
        ...,
        [ 0.0290,  0.0094,  0.0014,  ...,  0.0097, -0.0104, -0.0270],
        [-0.0008, -0.0201, -0.0252,  ...,  0.0176,  0.0107, -0.0357],
        [ 0.0006, -0.0253,  0.0301,  ...,  0.0320, -0.0110,  0.0161]],
       requires_grad=True)
Final weights -  tensor([[ 0.0010,  0.0010,  0.0010,  ...,  0.0010,  0.0010,  0.0010],
        [-0.0004, -0.0004, -0.0004,  ..., -0.0004, -0.0004, -0.0004],
        [-0.0014, -0.0014, -0.0014,  ..., -0.0014, -0.0014, -0.0014],
        ...,
        [ 0.0006,  0.0006,  0.0006,  ...,  0.0006,  0.0006,  0.0006],
        [ 0.0014,  0.0014,  0.0014,  ...,  0.0014,  0.0014,  0.0014],
        [-0.0003, -0.0003, -0.0003,  ..., -0.0003, -0.0003, -0.0003]])
